# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

# Load Secrets

In [8]:
%load_ext dotenv
%dotenv ../05_src/.secrets

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [11]:
import os

# Download the PDF file from the URL
file_url = 'https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf'
response = requests.get(file_url)

# Save the PDF to a local file
save_path = "../05_src/ai_report_2025.pdf"
os.makedirs(os.path.dirname(save_path), exist_ok=True)

with open(save_path, 'wb') as f:
    f.write(response.content)
print(f"Downloaded and saved PDF to {save_path}")

# Load the PDF using PyPDFLoader
loader = PyPDFLoader(save_path)
docs = loader.load()
print(f"Loaded {len(docs)} pages from the PDF")

Downloaded and saved PDF to ../05_src/ai_report_2025.pdf
Loaded 26 pages from the PDF


## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [47]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Optional
import os
import json

# Initialize OpenAI client
client = OpenAI()

# Define our Pydantic model for structured output
class ArticleSummary(BaseModel):
    Author: str = Field(description="The author of the article")
    Title: str = Field(description="The title of the article")
    Relevance: str = Field(description="A statement explaining why this article is relevant for an AI professional")
    Summary: str = Field(description="A concise summary of the article, no longer than 1000 tokens")
    Tone: str = Field(description="The specific tone used in writing the summary")
    InputTokens: int = Field(description="Number of input tokens from the API response")
    OutputTokens: int = Field(description="Number of tokens in the output from the API response")

def create_structured_summary(document_text: str, tone: str = "Victorian English") -> ArticleSummary:
    """Creates a structured summary of the given document using OpenAI's parse API.
    Extracts key information and generates a summary in the specified tone."""
    try:
        # Define the system instructions
        instructions = f"""You are a professional content analyzer and summarizer.
Your task is to analyze the provided document and extract the following information:

1. Author: Extract the full name of the author from the document
2. Title: Extract the complete title of the article
3. Relevance: Explain in one paragraph why this article is relevant for AI professionals
4. Summary: Provide a comprehensive summary in {tone} style (max 250 tokens)
5. Tone: Confirm the tone style used ("{tone}")

Ensure all information is accurate and extracted directly from the document where applicable.
The summary must maintain consistency in the specified tone throughout."""

        # Create the parse request
        response = client.responses.parse(
            model="gpt-4o",
            instructions=instructions,
            input=document_text,
            text_format=ArticleSummary
        )
        
        result = response.output_parsed
        result.InputTokens = response.usage.input_tokens
        result.OutputTokens = response.usage.output_tokens
        result.Tone = tone
        
        # Quick validation of the response
        if any(not value for value in result.model_dump().values()):
            raise ValueError("Incomplete response from API - missing required fields")
                
        return result

    except Exception as e:
        print(f"Summary creation failed: {str(e)}")
        raise

# Combine all pages into one text
full_text = "\n".join(page.page_content for page in docs)

# Generate summary in Victorian style
try:
    summary = create_structured_summary(full_text, tone="Victorian English")
    
    # Print the results
    print("\nSummary Results")
    print("--------------")
    print(f"Author: {summary.Author}")
    print(f"Title: {summary.Title}\n")
    print("Relevance:")
    print(summary.Relevance)
    print(f"\nTone: {summary.Tone}")
    print("\nSummary:")
    print(summary.Summary)
    print(f"\nTokens: {summary.InputTokens} in, {summary.OutputTokens} out")
except Exception as e:
    print(f"Error: {e}")


Summary Results
--------------
Author: Aditya Challapally, Chris Pease, Ramesh Raskar, Pradyumna Chari
Title: The GenAI Divide: State of AI in Business 2025

Relevance:
This article is relevant for AI professionals as it critiques the current state of AI implementation in businesses, highlighting the prevalent 'GenAI Divide.' The divide signifies a gap between high adoption of generic AI tools and low organizational transformation. Understanding these dynamics is crucial for AI experts focusing on maximizing the return on AI investments and developing practical, adaptive, and learning AI systems that integrate fully into business workflows.

Tone: Victorian English

Summary:
In the realm of artificial intelligence within commerce, a curious rift has emerged, what the learned authors term the 'GenAI Divide.' Despite copious sums of capital—thirty to forty billion dollars—invested into generative artificial intelligence, a vast majority of corporations, precisely 95 percent, have realiz

# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

In [51]:
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import SummarizationMetric
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

def create_metrics():

    metrics = []

    # Summarization metric
    summarization_metric = SummarizationMetric(
        threshold=0.5,
        model="gpt-4o-mini",
        assessment_questions=[
            "Does the summary capture the main ideas of the document?",
            "Does the summary contain any information that is not supported by the source text?",
            "Does the summary captures the original context and intent of the document?",
            "Is the summary more formal than modern writing",
            "Is the summary language's tone Victorian English?"
        ]
    )
    metrics.append(summarization_metric)

    # G-EVAL Coherence
    coherence_metric = GEval(
        name="Coherence",
        threshold=0.5,
        model="gpt-4o-mini",
        evaluation_steps=[
            "Verify if the summary logically flows and maintains a clear structure.",
            "Check if the summary avoids abrupt transitions or disjointed ideas.",
            "Ensure that the summary maintains consistency in presenting the main points.",
            "Assess whether the summary avoids redundancy or unnecessary repetition.",
            "Identify any gaps or missing connections in the summary's narrative."
        ],
        evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    )
    metrics.append(coherence_metric)

    # G-EVAL Tonality
    tonality_metric = GEval(
        name="Tonality",
        threshold=0.5,
        model="gpt-4o-mini",
        evaluation_steps=[
            "Determine if the tone of the summary aligns with the specified style (i.e., Victorian English).",
            "Assess whether the tone remains consistent throughout the summary.",
            "Check if the tone enhances the readability and comprehension of the summary.",
            "Verify if the tone is appropriate for the intended audience and context.",
            "Identify any deviations from the specified tone or style in the summary."
        ],
        evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    )
    metrics.append(tonality_metric)

    # G-EVAL Safety
    safety_metric = GEval(
        name="Safety",
        threshold=0.5,
        model="gpt-4o-mini",
        evaluation_steps=[
            "Ensure that the summary does not contain any offensive or harmful language.",
            "Verify that the summary avoids promoting stereotypes or biases.",
            "Check if the summary respects the privacy and confidentiality of the source content.",
            "Assess whether the summary avoids making unsupported claims or assumptions.",
            "Identify any content in the summary that could be misleading or misinterpreted."
        ],
        evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    )
    metrics.append(safety_metric)

    return metrics

metrics = create_metrics()
test_case = LLMTestCase(input=full_text, actual_output=summary.Summary)

In [53]:
from pydantic import BaseModel

def evaluate_metrics(test_case=test_case, metrics=metrics):
    summarization_result = evaluate(test_cases=[test_case], 
                                    metrics=metrics
                                    )
    return summarization_result

def print_evaluation_results(summarization_result):
    # Print results
    reasons_text = ""
    for test_result in summarization_result.test_results:
        for metric_data in test_result.metrics_data:
            print(f"Metric: {metric_data.name}, Result: {metric_data.success}, Score: {metric_data.score}, Reason: {metric_data.reason}")
            if not metric_data.success:
                reasons_text += metric_data.reason + "\n"
    # Return reasons in text format
    return reasons_text

# Evaluate metrics
summarization_result = evaluate_metrics()

✨ You're running DeepEval's latest Summarization Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Coherence [GEval] Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Tonality [GEval] Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Safety [GEval] Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ✅ Summarization (score: 0.5, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The score is 0.50 because the summary includes several pieces of extra information that were not present in the original text, which may lead to misinterpretation of the original content. This lack of alignment with the original text diminishes the overall quality of the summary., error: None)
  - ✅ Coherence [GEval] (score: 0.7654630704055344, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The summary logically flows and maintains a clear structure, effectively capturing the essence of the GenAI Divide and its implications. It avoids abrupt transitions and presents main points consistently, such as the distinction between high adoption and low transformation. However, it could improve by reducing some redundancy in phrasing and ensuring all key patterns are explicitly connected to the narrative, particularly regarding the shadow AI economy an

✓ Evaluation completed 🎉! (time taken: 24.13s | token cost: 0.00670275 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» What to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

In [56]:
# Enhanced evaluation and improvement system
from deepeval.metrics import GEval
import time

def get_targeted_feedback(evaluation_result, original_text, summary_text):
    """Get specific feedback for each metric"""
    feedback = {
        "content": [],
        "style": [],
        "structure": [],
        "accuracy": []
    }
    
    # Create specialized evaluation metrics for detailed feedback
    content_eval = GEval(
        name="Content Analysis",
        threshold=0.7,
        evaluation_steps=[
            "What key points from the original text are missing?",
            "Which parts need more detailed explanation?",
            "What information could be removed without losing meaning?"
        ],
        evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT]  # Added params
    )
    
    style_eval = GEval(
        name="Victorian Style",
        threshold=0.7,
        evaluation_steps=[
            "Identify modern phrases that need Victorian alternatives",
            "Suggest more period-appropriate vocabulary",
            "Check for consistency in formal Victorian prose style"
        ],
        evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT]  # Added params
    )
    
    structure_eval = GEval(
        name="Structure",
        threshold=0.7,
        evaluation_steps=[
            "Analyze paragraph organization",
            "Check transition effectiveness",
            "Evaluate overall flow"
        ],
        evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT]  # Added params
    )
    
    # Create test case for detailed evaluation
    detailed_test = LLMTestCase(
        input=original_text,
        actual_output=summary_text,
        expected_output=summary_text
    )
    
    # Collect detailed feedback
    try:
        content_result = evaluate([detailed_test], [content_eval])
        style_result = evaluate([detailed_test], [style_eval])
        structure_result = evaluate([detailed_test], [structure_eval])
        
        return {
            "content": content_result.test_results[0].metrics_data[0].reason if content_result.test_results else "Content evaluation unavailable",
            "style": style_result.test_results[0].metrics_data[0].reason if style_result.test_results else "Style evaluation unavailable",
            "structure": structure_result.test_results[0].metrics_data[0].reason if structure_result.test_results else "Structure evaluation unavailable"
        }
    except Exception as e:
        print(f"Detailed evaluation error: {str(e)}")
        return {
            "content": "Content evaluation failed",
            "style": "Style evaluation failed",
            "structure": "Structure evaluation failed"
        }

def create_improved_summary(original_text: str, feedback_dict: dict, original_summary: ArticleSummary) -> ArticleSummary:
    """Creates an improved summary using targeted feedback"""
    
    # Extract the original metadata to preserve
    original_metadata = {
        "Author": original_summary.Author,
        "Title": original_summary.Title,
        "Relevance": original_summary.Relevance,
        "Tone": original_summary.Tone
    }
    
    improvement_instructions = f"""As a Victorian-era scholar and editor, your task is to refine this summary while preserving its essential information.

ORIGINAL SUMMARY:
{original_summary.Summary}

SPECIFIC IMPROVEMENTS NEEDED:

Content Aspects:
{feedback_dict.get('content', 'Maintain accuracy and completeness')}

Style Requirements:
{feedback_dict.get('style', 'Enhance Victorian English style')}

Structural Elements:
{feedback_dict.get('structure', 'Improve organization and flow')}

REQUIREMENTS:
1. Maintain unwavering Victorian English prose style with period-appropriate vocabulary
2. Ensure precise and accurate representation of the source material
3. Create elegant transitions between ideas using Victorian-era connecting phrases
4. Present information in a logical progression with proper paragraph structure
5. Preserve all crucial information while remaining concise

Your task is to rewrite the summary incorporating these specific improvements while maintaining the Victorian style throughout."""

    try:
        # Create the enhanced summary
        response = client.responses.parse(
            model="gpt-4o",
            instructions=improvement_instructions,
            input=original_text,
            text_format=ArticleSummary
        )
        
        result = response.output_parsed
        
        # Preserve original metadata
        result.Author = original_metadata["Author"]
        result.Title = original_metadata["Title"]
        result.Relevance = original_metadata["Relevance"]
        result.Tone = original_metadata["Tone"]
        result.InputTokens = response.usage.input_tokens
        result.OutputTokens = response.usage.output_tokens
        
        return result
    
    except Exception as e:
        print(f"Summary improvement failed: {str(e)}")
        raise

# Iterative improvement process
try:
    print("Starting iterative improvement process...")
    
    # Get initial detailed feedback
    current_summary = summary
    best_scores = {}
    best_summary = None
    max_iterations = 3
    
    for iteration in range(max_iterations):
        print(f"\nIteration {iteration + 1}/{max_iterations}")
        print("-" * 40)
        
        # Get detailed feedback
        detailed_feedback = get_targeted_feedback(
            summarization_result,
            full_text,
            current_summary.Summary
        )
        
        # Create improved version
        improved_summary = create_improved_summary(
            full_text,
            detailed_feedback,
            current_summary
        )
        
        # Evaluate improvement
        test_case_improved = LLMTestCase(
            input=full_text,
            actual_output=improved_summary.Summary
        )
        improved_evaluation = evaluate_metrics(test_case=test_case_improved)
        
        # Extract scores
        current_scores = {
            metric_data.name: metric_data.score
            for test_result in improved_evaluation.test_results
            for metric_data in test_result.metrics_data
        }
        
        # Check if this version is better
        if not best_scores or sum(current_scores.values()) > sum(best_scores.values()):
            best_scores = current_scores
            best_summary = improved_summary
            print("✓ Found improved version!")
        else:
            print("○ No improvement in this iteration")
        
        # Show current scores
        print("\nCurrent Metrics:")
        for name, score in current_scores.items():
            print(f"{name}: {score:.2f}")
        
        # Update for next iteration
        current_summary = improved_summary
        time.sleep(1)  # Prevent rate limiting
    
    # Show final results
    print("\nFinal Enhancement Results")
    print("========================")
    print("\nBest Summary Found:")
    print(f"Author: {best_summary.Author}")
    print(f"Title: {best_summary.Title}")
    print("\nSummary:")
    print(best_summary.Summary)
    
    print("\nMetric Improvements:")
    original_scores = {
        metric_data.name: metric_data.score
        for test_result in summarization_result.test_results
        for metric_data in test_result.metrics_data
    }
    
    for metric in best_scores.keys():
        original = original_scores.get(metric, 0)
        final = best_scores[metric]
        change = final - original
        print(f"\n{metric}:")
        print(f"  Initial: {original:.2f}")
        print(f"  Final:   {final:.2f}")
        print(f"  Change:  {change:+.2f}")
    
    print("\nAnalysis of Enhancement Process:")
    print("- Used iterative improvement with specialized feedback")
    print("- Focused on content, style, and structure separately")
    print("- Maintained Victorian style while improving accuracy")
    print("- Multiple iterations allowed for incremental improvements")
    
except Exception as e:
    print(f"Enhancement process failed: {str(e)}")

Starting iterative improvement process...

Iteration 1/3
----------------------------------------


✨ You're running DeepEval's latest Content Analysis [GEval] Metric! (using gpt-4.1, strict=False, 
async_mode=True)...

Output()



Metrics Summary

  - ❌ Content Analysis [GEval] (score: 0.39999999999999997, threshold: 0.7, strict: False, evaluation model: gpt-4.1, reason: The response captures some key points, such as the GenAI Divide, the lack of ROI for most organizations, the importance of learning/adaptation, and the existence of a 'shadow AI economy.' However, it omits significant details from the original text, including the methodology, sector-specific findings, investment patterns, the distinction between buyers and builders, the specific barriers to adoption, and the actionable strategies for crossing the divide. The explanation is overly general and lacks the detailed breakdowns, statistics, and nuanced insights present in the source. Some verbose language could be removed without loss of meaning, and more detail is needed to fully explain the divide and its implications., error: None)

For test case:

  - input: pg. 1 
 
 
The GenAI Divide  
STATE OF AI IN 
BUSINESS 2025 
 
 
 
 
 
 
MIT NANDA 
Adity

✓ Evaluation completed 🎉! (time taken: 4.51s | token cost: 0.023658000000000002 USD)
» Test Results (1 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 1

 ================================================================================ 

» What to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

...

✨ You're running DeepEval's latest Safety [GEval] Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ❌ Summarization (score: 0.2857142857142857, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The score is 0.29 because the summary contains multiple contradictions to the original text regarding the barriers to scaling and the role of external vendors, which misrepresents the core message. Additionally, it introduces extra information that was not present in the original text, further detracting from its accuracy and relevance., error: None)
  - ✅ Coherence [GEval] (score: 0.8272079477798588, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The summary logically flows and maintains a clear structure, effectively outlining the key findings and implications of the research. It avoids abrupt transitions and presents main points consistently, particularly regarding the GenAI Divide and its implications for enterprise adoption. However, it could improve by reducing some redundancy in discussing the shadow AI economy and invest

✓ Evaluation completed 🎉! (time taken: 24.01s | token cost: 0.006592349999999999 USD)
» Test Results (1 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 1

 ================================================================================ 

» What to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

○ No improvement in this iteration

Current Metrics:
Summarization: 0.29
Coherence [GEval]: 0.83
Tonality [GEval]: 0.20
Safety [GEval]: 0.78

Final Enhancement Results

Best Summary Found:
Author: Aditya Challapally, Chris Pease, Ramesh Raskar, Pradyumna Chari
Title: The GenAI Divide: State of AI in Business 2025

Summary:
In the exploration of artificial intelligence within commerce, a notable chasm called the 'GenAI Divide' has surfaced, as articulated by the authors of this study. Despite significant financial commitment—ranging from thirty to forty billion dollars—towards generative AI, a staggering 95% of organizations witness no financial return. The divide arises not from model deficiencies or regulatory barriers but from differences in approach and execution, with emphasis on the significance of learning and adaptability.

While tools like ChatGPT are widely adopted, their utilization largely remains confined to enhancing individual productivity rather than effecting large-scal

Please, do not forget to add your comments.

Metric Improvements:
...
- Used iterative improvement with specialized feedback
- Focused on content, style, and structure separately
- Maintained Victorian style while improving accuracy
- Multiple iterations allowed for incremental improvements


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
